# RoundRobinGroupChat
RoundRobinGroupChat は、すべてのエージェントが同じコンテキストを共有し、ラウンドロビン方式で順番に返信するシンプルなながらも効果的なチーム構成です。各エージェントは、自分の番が回ってきた際に、他のすべてのエージェントに返信をブロードキャストし、チーム全体が一致したコンテキストを維持します。


In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[mcp,openai,azure]"

In [ ]:
import logging  
from typing import Any, List  
import os  
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import RoundRobinGroupChat  # keeps implementation simple & familiar  
from autogen_agentchat.conditions import TextMessageTermination  
from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
  
load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Tools: {tools}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## エージェント定義

In [ ]:

# 3. -----------------  Agent Definitions -----------------  
analysis_planning_agent = AssistantAgent(  
    name="analysis_planning",  
    model_client=model_client,  
    tools=tools,  
    system_message=(  
"""
あなたは「分析 & 計画エージェント（Analysis & Planning Agent）」です – 全体のオーケストレーターとして機能します。

あなたの役割:
1) 顧客からの抽象的なリクエストを解析すること。
2) リクエストを明確なサブタスクに分解し、それぞれを専門エージェント
   （crm_billing、product_promotions）に割り当てること。
3) 各専門エージェントからの出力を統合し、1つの包括的で一貫性のある
   顧客向けの回答としてまとめること。
4) 満足のいく結果が得られたら、以下のプレフィックスを付けて顧客に
   最終回答を返すこと: FINAL_ANSWER:

まだ情報が不足している場合は、専門エージェントとの対話を継続してください。
情報が揃っていれば、最終回答で終了してください。
"""
    ),  
)  

crm_billing_agent = AssistantAgent(  
    name="crm_billing",  
    model_client=model_client,  
    tools=tools,  
    system_message=(  
"""
あなたは「CRM & 請求エージェント（CRM & Billing Agent）」です。

- アカウント、サブスクリプション、請求書、支払い情報などを取得するために、
  構造化されたCRM／請求システムを照会します。
- 請求ポリシー、支払い処理、返金ルールなどに関する *ナレッジベース* 記事を確認し、
  回答が正確かつポリシーに準拠していることを保証します。
- 簡潔で構造化された情報を返答し、検出されたポリシー上の懸念事項があれば
  フラグを立てます。
"""
    ),  
)  

product_promotions_agent = AssistantAgent(  
    name="product_promotions",  
    model_client=model_client,  
    tools=tools,  
    system_message=(  
"""
あなたは「製品 & プロモーションエージェント（Product & Promotions Agent）」です。

- 構造化された情報源から、プロモーションのオファー、製品の在庫状況、
  適格条件、割引情報などを取得します。
- *ナレッジベース* のFAQ、利用規約、ベストプラクティスを活用して、
  回答を補足します。
- 事実に基づいた、最新の製品／プロモーション情報を提供します。
"""
    ),  
)  



In [ ]:
# 4. -----------------  Assemble Team -----------------  
# ラウンドロビンは簡単なデフォルト設定です：オーケストレーターを最後に配置し、
# 専門家が発言を終えた後に収集し完了させます。チャットは
# オーケストレーターが発言した時点で停止します（内容に関わらず）、
# テキストメッセージの終了はエージェント名でキーイングされているためです。 
participants: List[AssistantAgent] = [  
    crm_billing_agent,  
    product_promotions_agent,  
    analysis_planning_agent,      # orchestrator always concludes a cycle  
]  

## 停止条件
AutoGen には 無限ループを防止するため 8 つの組み込みの終了条件が定義されています。終了条件は以下のように OR 条件で指定できるのが便利です。

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html


In [ ]:
#エージェントによってテキストメッセージが生成された際に停止します。
termination_condition = TextMessageTermination("analysis_planning")

# RoundRobinGroupChat の実行

In [ ]:
team_agent = RoundRobinGroupChat(
    participants=participants,
    termination_condition=termination_condition,
)

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成

    task = "ユーザーID:123 の出荷状況を確認してください。"

    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成

    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "商品ID:339の商品詳細を教えて"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成

    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "SNS分析を行い、日付ごとのツイート数を集計してください"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成

    task = "2024年9月~10月の受注数を集計し、最も受注数が多いかった日付を教えて"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成

    task = "2024年9月~10月の受注数を集計し、さらにSNS分析を行い、日付ごとのツイート数を集計した結果何かわかることはあるか？"
    response = team_agent.run_stream(task=task)

    await Console(response)